In [1]:
# Импортируем библиотеки
import json
import requests
from pprint import pprint
import pandas as pd

In [2]:
# задаем токен
token = 'AQAAAABcBYlwAAeZHVaHxB8LnEjylNKZLg3Oldg'
headers = {'Authorization': 'OAuth ' + token}
# задаем параметра запроса
params = {'metrics': 'ym:s:visits,ym:s:robotPercentage',
          'dimensions': 'ym:s:lastsignTrafficSource',  
          'date1': '2020-09-01',  
          'date2': '2020-09-30',     
          'ids': 30177909,
          'accuracy':'full',
          'limit':100000}
# делаем запрос
response = requests.get('https://api-metrika.yandex.net/stat/v1/data', params=params, headers=headers)
print (response.status_code)

200


200 — значит, запрос был составлен корректно и API передал ответ

In [3]:
# запишем данные в переменную
metrika_data = response.json()
# преобразуем данные в так называемый плоский вид, то есть избавимся от вложенных словарей
def getMetrikaDataInListOfDicts(metrika_data):
    list_of_dicts = []
    dimensions_list = metrika_data['query']['dimensions']
    metrics_list = metrika_data['query']['metrics']
    for data_item in metrika_data['data']:
        d = {}
        for i,dimension in enumerate(data_item['dimensions']):
            d[dimensions_list[i]] = dimension['name']
        for i,metric in enumerate(data_item['metrics']):
            d[metrics_list[i]] = metric
        list_of_dicts.append(d)
    return list_of_dicts
metrika_list_of_dicts = getMetrikaDataInListOfDicts(metrika_data)

In [4]:
# создаем датафрейм
metrika_df = pd.DataFrame(metrika_list_of_dicts)
# переименуем столбцы
metrika_df.columns=['Source', 'Visits', 'Percentage of bots']
# расчет количества ботов
metrika_df['Number of bots'] = metrika_df['Visits']*metrika_df['Percentage of bots']/100
# округление данных 
metrika_df['Percentage of bots'] = metrika_df['Percentage of bots'].round(2)
metrika_df['Number of bots'] = metrika_df['Number of bots'].round(0)
print(metrika_df)

                          Source  Visits  Percentage of bots  Number of bots
0          Search engine traffic  3334.0               22.17           739.0
1                 Direct traffic   757.0               23.38           177.0
2                   Link traffic   126.0               24.60            31.0
3         Social network traffic    80.0               13.75            11.0
4               Internal traffic    10.0               30.00             3.0
5            Cached page traffic     6.0               16.67             1.0
6  Recommendation system traffic     2.0                0.00             0.0


Наибольшее количество ботов заходят на сайт из источника Search engine traffic (поисковый трафик) - 739. При этом максимальный процент ботов наблюдает из источника Internal traffic (внутренний трафик) - 30%.

In [5]:
# расчет количества ботов и их доли в общем трафике
sum_visits = metrika_df['Visits'].sum()
sum_bots = metrika_df['Number of bots'].sum()
proportion_bots = sum_bots*100/sum_visits
proportion_bots = proportion_bots.round(2)
print('Количество ботов, которые заходили на сайт: ', sum_bots)
print('Процент ботов (в общем трафике): ',proportion_bots)

Количество ботов, которые заходили на сайт:  962.0
Процент ботов (в общем трафике):  22.29


In [6]:
# задаем параметра запроса
params = {'metrics': 'ym:s:robotPercentage',
          'dimensions': 'ym:s:hour',  
          'date1': '2020-09-01',  
          'date2': '2020-09-30',     
          'ids': 30177909,
          'accuracy':'full',
          'limit':100000}
# делаем запрос
response = requests.get('https://api-metrika.yandex.net/stat/v1/data', params=params, headers=headers)
print (response.status_code)

200


200 — значит, запрос был составлен корректно и API передал ответ

In [7]:
# запишем данные в переменную
metrika_data = response.json()
# преобразуем данные в так называемый плоский вид, то есть избавимся от вложенных словарей
def getMetrikaDataInListOfDicts(metrika_data):
    list_of_dicts = []
    dimensions_list = metrika_data['query']['dimensions']
    metrics_list = metrika_data['query']['metrics']
    for data_item in metrika_data['data']:
        d = {}
        for i,dimension in enumerate(data_item['dimensions']):
            d[dimensions_list[i]] = dimension['name']
        for i,metric in enumerate(data_item['metrics']):
            d[metrics_list[i]] = metric
        list_of_dicts.append(d)
    return list_of_dicts
metrika_list_of_dicts = getMetrikaDataInListOfDicts(metrika_data)

In [8]:
# создаем датафрейм
metrika_df = pd.DataFrame(metrika_list_of_dicts)
# переименуем столбцы
metrika_df.columns=['Hour','Percentage of bots']
# округление данных по процентам ботов
metrika_df['Percentage of bots'] = metrika_df['Percentage of bots'].round(2)
print(metrika_df)

     Hour  Percentage of bots
0   13:00               29.30
1   17:00               25.19
2   09:00               25.12
3   11:00               24.82
4   12:00               24.75
5   14:00               24.67
6   16:00               24.19
7   15:00               23.91
8   19:00               22.97
9   08:00               22.64
10  10:00               21.25
11  22:00               21.05
12  18:00               20.71
13  23:00               19.11
14  21:00               18.45
15  04:00               18.18
16  20:00               17.73
17  01:00               17.57
18  00:00               16.52
19  03:00               15.79
20  06:00               15.09
21  07:00               14.12
22  02:00               12.50
23  05:00                9.52


Наибольший процент ботов заходит на сайт в дневное и утренее время суток. Максимальное значение соответствует 13:00